# Bayesian losses module

This tutorial demonstrates the functionality of a Bayesian losses module. It covers basic operations such as checking trainable parameters and forward propagation of loss functions.


## Libraries

To get started, you'll need to import some essential libraries. The specific libraries you use will depend on the backend you've chosen, such as PyTorch, TensorFlow, or Jax. Additionally, you'll need to import NumPy.

In [ ]:
import torch
import numpy as np
import tensorflow as tf

## Functions

The ``check_parameters`` function verifies the existence of trainable parameters.

In [2]:
def check_parameters():

    print("Check the existence of trainable parameters in the classes...")

    torch_list_parameters = list(torch_module.parameters())
    assert (
        len(torch_list_parameters) != 0
    ), "No parameters availables in TorchTestModule"

    trainable_variables = tf_module.trainable_variables
    assert (
        len(trainable_variables) != 0
    ), "No trainable parameters available in TFTestModule"

    print("Test passed!", "\n\n")

The ``check_forward_losses`` function ensures that the loss functions produce consistent outputs.

In [3]:
def check_forward_losses():

    print("Check the forward propagation of the loss functions...")

    # Input data
    input_data = np.random.randn(1, 10).astype(np.float32)
    y_true = np.random.randn(1, 10).astype(np.float32)
    y_pred = np.random.randn(1, 10).astype(np.float32)

    # PyTorch forward pass
    torch_input = torch.from_numpy(input_data)
    torch_output = torch_module(torch_input)
    torch_kl_divengence_output = torch_kl_divengence(torch_module)
    torch_elbo_loss_output = torch_elbo_loss(
        torch.from_numpy(y_true), torch.from_numpy(y_pred), torch_module
    )

    # Tensorflow forward pass
    tf_input = tf.convert_to_tensor(input_data)
    tf_output = tf_module(tf_input)
    tf_kl_divergence_output = tf_kl_divengence(tf_module)
    tf_elbo_loss_output = tf_elbo_loss(
        tf.convert_to_tensor(y_true), tf.convert_to_tensor(y_pred), tf_module
    )

    # Assert that the outputs are similar
    print("Torch output:", torch_output)
    print("TensorFlow output:", tf_output)
    print("Outputs are similar:", np.allclose(torch_output.detach().numpy(), tf_output))

    print("Torch KL divergence output:", torch_kl_divengence_output)
    print("TensorFlow KL divergence output:", tf_kl_divergence_output)
    print(
        "KL divergence outputs are similar:",
        np.allclose(
            torch_kl_divengence_output.detach().numpy(), tf_kl_divergence_output
        ),
    )

    print("Torch ELBO loss output:", torch_elbo_loss_output)
    print("TensorFlow ELBO loss output:", tf_elbo_loss_output)
    print(
        "ELBO loss outputs are similar:",
        np.allclose(torch_elbo_loss_output.detach().numpy(), tf_elbo_loss_output),
    )

    print("Test passed!", "\n\n")

The ``run_all_tests`` function executes all test functions in sequence to validate the module's functionality.

In [4]:
def run_all_tests():

    check_parameters()
    check_forward_losses()

## Illia

When setting the backend, we import the Illia library, which provides Bayesian module implementations. Note that backend selection requires a kernel restart and cannot be changed dynamically.

In [5]:
from illia.torch.nn.base import BayesianModule as TorchBayesianModule
from illia.tf.nn.base import BayesianModule as TFBayesianModule
from illia.torch.nn.losses import (
    KLDivergenceLoss as TorchKLDivergenceLoss,
    ELBOLoss as TorchELBOLoss,
)
from illia.torch.nn.linear import Linear as TorchLinear

from illia.tf.nn.losses import (
    KLDivergenceLoss as TFKLDivergenceLoss,
    ELBOLoss as TFELBOLoss,
)
from illia.tf.nn.linear import Linear as TFLinear

## Class definitions

Define test classes implementing a simple linear layer and a method to compute KL divergence. These classes will be used in the tests.

In [ ]:
class TorchTestModule(TorchBayesianModule):

    def __init__(self):

        super().__init__()

        self.linear = TorchLinear(10, 5)

    def forward(self, x):

        return self.linear(x)

    def kl_cost(self):

        return torch.tensor(1.0), 1


class TFTestModule(TFBayesianModule):

    def __init__(self):

        super().__init__()

        self.linear = TFLinear(10, 5)

    def call(self, x):

        return self.linear(x)

    def kl_cost(self):

        return tf.constant(1.0), 1


# PyTorch
torch_kl_divengence = TorchKLDivergenceLoss()
torch_elbo_loss = TorchELBOLoss(loss_function=torch.nn.MSELoss())
torch_module = TorchTestModule()

# Tensorflow
tf_kl_divengence = TFKLDivergenceLoss()
tf_elbo_loss = TFELBOLoss(loss_function=tf.keras.losses.MeanSquaredError())
tf_module = TFTestModule()

Finally, run all tests to ensure that the module's functionalities work as expected across backends.

In [ ]:
run_all_tests()